In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
geneNamesPath = "/scratch/indikar_root/indikar1/shared_data/directRNA/references/geneNames.csv"

gf = pd.read_csv(geneNamesPath)
print(f"{gf.shape=}")
print(gf['geneId'].nunique())

gf = gf[['geneId', 'geneName']].drop_duplicates()
print(f"{gf.shape=}")

gf.head()

gf.shape=(262041, 5)
41407
gf.shape=(41407, 2)


,geneId,geneName
0,ENSG00000160072,ATAD3B
7,ENSG00000225972,MTND1P23
9,ENSG00000224315,RPL7P7
11,ENSG00000198744,MTCO3P12
13,ENSG00000279928,DDX11L17


In [7]:
dirPath = "/scratch/indikar_root/indikar1/shared_data/directRNA/counts/"

df = gf.copy()

fileNames = []

for f in os.listdir(dirPath):
    if "poly" in f:
        fileId = f.split(".")[0]
        fullPath = f"{dirPath}{f}"
        tf = pd.read_csv(fullPath, header=None, sep="\t")
        tf.columns = ['geneId', fileId]
        fileNames.append(fileId)

        df = pd.merge(df, tf, 
                      how='left',
                      left_on='geneId',
                      right_on='geneId')
        
        break
    
print(f"{df.shape=}")
df = df.fillna(0)
df.head()

df.shape=(41407, 3)


,geneId,geneName,poly_FAR96055_pass_8dd551a2_23
0,ENSG00000160072,ATAD3B,0
1,ENSG00000225972,MTND1P23,0
2,ENSG00000224315,RPL7P7,0
3,ENSG00000198744,MTCO3P12,0
4,ENSG00000279928,DDX11L17,0


In [8]:
df['GeneCount'] = df[fileNames].sum(axis=1)

df['GeneCount'].max()

0

In [6]:
for f in fileNames:
    print(f, df[f].max())

total_FAS69398_pass_14a3cc95_9 27
